# <center> CNN on MNIST dataset </center>

### Set Current working directory

In [1]:
import os

# Change directory
PATH = os.getcwd() 
print (PATH)
os.chdir(PATH)

C:\Users\Gunjan\Downloads\CNN-master\CNN-master


### Keras CNN layers

- Import the CNN layers - convolutional, pooling from Keras.

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score, confusion_matrix

Using TensorFlow backend.


### Load image data from MNIST.

- MNIST database of handwritten digits
- Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

Returns 2 tuples :

- x_train, x_test : uint8 array of grayscale image data with shape (num_samples, 28, 28).
- y_train, y_test : uint8 array of digit labels (integers in range 0-9) with shape (num_samples,).

Arguments:

- path : if you do not have the index file locally (at '~/.keras/datasets/' + path), it will be downloaded to this location.


In [3]:
from keras.datasets import mnist
 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


### Reshape input data

- When using the tensorflow backend, you must explicitly declare a dimension for the depth of the input image. E.g. a full-color image with all 3 RGB channels will have a depth of 3.

- Our MNIST images only have a depth of 1, but we must explicitly declare that. In other words, we want to transform our dataset from having shape (n, width, height) to (n, width, height, channel).

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

print(X_train.shape, X_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


### Normalize the data to the range [0, 1]

- Max value X_train can take is 255, so it is divided by 255

In [5]:
X_train = X_train/255
X_test = X_test/255

### Preprocess Class Labels

We have 10 different classes, one for each digit, but its a 1-dimensional array.

In [6]:
print(y_train[:10])

[5 0 4 1 9 2 1 3 1 4]


#### Convert 1-dimensional class arrays to 10-dimensional class matrices

In [7]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

print(Y_train.shape, Y_test.shape)

(60000, 10) (10000, 10)


In [8]:
print(Y_train[:10])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


### Define Model Architecture

- First parameters correspond to the number of convolution filters 

- Next parameter correspond to kernal size

- When using this layer as the first layer in a model, provide the keyword argument input_shape. e.g. input_shape = (28, 28, 1) for 28x28 gray pictures in data_format = "channels_last" i.e. `(batch, height, width, channels)`.

This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. 

In [9]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (28, 28, 1), activation = 'relu'))

#### Add more layers to our model

- MaxPooling2D 

Is a way to reduce the number of parameters in our model by sliding a 2x2 pooling filter across the previous layer and taking the max of the 4 values in the 2x2 filter.
    
- Dropout

This is a method for regularizing our model in order to prevent overfitting. 

In [10]:
model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

So far, for model parameters, we've added two Convolution layers. To complete our model architecture, let's add a fully connected layer and then the output layer.

For Dense layers, the first parameter is the output size of the layer. 

Note :

- The final layer has an output size of 10, corresponding to the 10 classes of digits.

- The weights from the Convolution layers must be flattened (made 1-dimensional) before passing them to the fully connected Dense layer.


In [11]:
model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

#### Compile the model by providing the loss function and the optimizer 

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Fit model on training data

In [14]:
model.fit(X_train, Y_train, batch_size = 32, epochs = 2)

Epoch 1/2
60000/60000 [==============================] - 122s 2ms/step - loss: 0.2196 - acc: 0.9341
Epoch 2/2
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0907 - acc: 0.9732


#### Evaluate the model on test data

In [15]:
model.evaluate(X_test, Y_test)

10000/10000 [==============================] - 4s 448us/step


[0.043278335338679606, 0.9864]

#### Make predictions on test data 

In [16]:
model.predict(X_test)

array([[5.94194000e-11, 2.81742873e-09, 1.07254465e-07, ...,
        9.99999762e-01, 8.87982656e-12, 7.58500729e-09],
       [2.41588971e-08, 6.58998488e-07, 9.99999285e-01, ...,
        2.90722696e-10, 1.98452561e-08, 1.92022765e-12],
       [3.83643295e-09, 9.99979377e-01, 6.65957305e-06, ...,
        1.05651443e-05, 3.43210587e-07, 5.51095312e-08],
       ...,
       [6.45984863e-11, 6.11097519e-08, 4.26024049e-10, ...,
        2.50747050e-08, 1.89021091e-06, 2.43795898e-06],
       [2.68572376e-06, 4.01828126e-09, 7.57522933e-09, ...,
        4.60187096e-07, 2.97798845e-03, 1.96819670e-07],
       [4.25171578e-08, 3.53370319e-11, 1.06352149e-09, ...,
        1.87071888e-13, 5.74799153e-10, 2.50069063e-11]], dtype=float32)

#### Calculate Prediction Accuracy

In [19]:
y_pred_class = model.predict_classes(X_test)
accuracy_score(y_test, y_pred_class)

0.9864

#### Print the Confusion Matrix

In [20]:
confusion_matrix(y_test, y_pred_class)

array([[ 976,    0,    1,    0,    0,    0,    1,    1,    1,    0],
       [   0, 1129,    2,    2,    0,    1,    1,    0,    0,    0],
       [   1,    0, 1025,    0,    1,    0,    0,    4,    1,    0],
       [   1,    0,    3, 1001,    0,    2,    0,    2,    1,    0],
       [   1,    1,    2,    0,  970,    0,    3,    1,    2,    2],
       [   2,    0,    0,    7,    0,  875,    4,    0,    4,    0],
       [   7,    2,    0,    1,    1,    2,  944,    0,    1,    0],
       [   0,    2,   12,    1,    0,    0,    0, 1012,    1,    0],
       [   8,    0,    2,    1,    0,    0,    1,    2,  958,    2],
       [   6,    3,    0,    1,    8,    3,    0,    6,    8,  974]])

Reference: 

    https://keras.io/
    https://elitedatascience.com
    http://scikit-learn.org/stable/modules/classes.html